In [ ]:
from datetime import datetime
from ray.job_submission import JobSubmissionClient
import pandas as pd

client = JobSubmissionClient("http://localhost:8265")
df = pd.DataFrame([i.__dict__ for i in client.list_jobs()])
# df = df[df["status"]=="RUNNING"]
df["start_time"] = df["start_time"].apply(
    lambda x: (
        datetime.fromtimestamp(x / 1000).strftime("%Hh%M %d/%m/%Y")
        if x is not None
        else None
    )
)
# df["end_time"] = df["end_time"].apply(lambda x: datetime.fromtimestamp(x/1000) if not np.isnan(x) else x)
df["who"] = df.runtime_env.apply(
    lambda x: (
        x["env_vars"]["USER"] if "env_vars" in x and "USER" in x["env_vars"] else None
    )
)
# df = df.loc[:,['type', 'job_id', 'submiss
# ion_id', 'status', 'entrypoint', 'message', 'start_time','end_time','who']]
df = df[~df.submission_id.isna()]
df = df[df["status"] == "RUNNING"]
df.reset_index(inplace=True, drop=True)
df

In [ ]:
import time
from ray.job_submission import JobSubmissionClient, JobStatus
from datetime import datetime
from mum.utils.generate_runtime_env import generate_runtime_saml

runtime_env = generate_runtime_saml(
    profile="automl",
    to_yaml=False,
    workdir=True,
    inference_runtime=False,
    path_workdir="/Users/andresospina/Workspace/mls-mum-project",
)

client = JobSubmissionClient("http://localhost:8265")

job_id = client.submit_job(
    submission_id="_".join(["download_pandas10M", datetime.now().strftime("%H_%M_%S")]),
    # Entrypoint shell command to execute
    entrypoint=f"python download_pandas70M.py",
    # Path to the local directory that contains the script.py file
    runtime_env=runtime_env,
)

